NOTE: I am defining the gate time as the shortest `total_time` where the error gets below 10^-3 within 1000 iterations of QOC.

I determined this shortest time by binary searching. This notebook just shows the final `total_time` found for each gate.

---
## Setup

In [1]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

/software/python_ucs4-2.7.12-el7-x86_64/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
import re, os, h5py, sys, math
from datetime import datetime

path = '/project/ftchong/qoc/pranav/'
assert False, 'make a directory in /project/ftchong/qoc and update the path above. Also update file_name to current date.\
               Then delete this assert'
file_name = datetime.today().strftime('%h%d')

Populating the interactive namespace from numpy and matplotlib


In [3]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 1000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}

----
## H Gate: 2.1 ns

In [4]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = np.array([[1, 1], [1, -1]]) / math.sqrt(2)
U = transmon_gate(U, d)

In [10]:
total_time = 2.1
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path = path)

data saved at: /project/ftchong/qoc/pranav/00084_Feb09.h5
Using 7 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :5.04e-01; Runtime: 0.1s; Iterations = 0, grads =   2.461e-02, unitary_metric = 1.00000
Error = :1.19e-02; Runtime: 1.2s; Iterations = 100, grads =   1.287e-04, unitary_metric = 1.00000
Error = :4.33e-03; Runtime: 2.3s; Iterations = 200, grads =   1.723e-05, unitary_metric = 1.00000
Error = :5.95e-03; Runtime: 3.4s; Iterations = 300, grads =   2.655e-05, unitary_metric = 0.99999
Error = :1.07e-02; Runtime: 4.4s; Iterations = 400, grads =   1.004e-04, unitary_metric = 1.00000
Error = :1.02e-02; Runtime: 5.5s; Iterations = 500, grads =   1.574e-04, unitary_metric = 1.00000
Error = :4.92e-03; Runtime: 

---
## CNOT Gate: 7.1 ns

In [11]:
N = 2
connected_qubit_pairs = [(0, 1)]
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
U = transmon_gate(U, d)

In [20]:
total_time = 7.1
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path = path)

data saved at: /project/ftchong/qoc/pranav/00093_Feb09.h5
Using 4 Taylor terms and 5 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :8.45e-01; Runtime: 0.1s; Iterations = 0, grads =   1.937e-02, unitary_metric = 1.00002
Error = :3.72e-02; Runtime: 4.3s; Iterations = 100, grads =   2.358e-04, unitary_metric = 0.99996
Error = :1.35e-02; Runtime: 8.3s; Iterations = 200, grads =   6.055e-05, unitary_metric = 0.99996
Error = :5.00e-03; Runtime: 12.3s; Iterations = 300, grads =   1.641e-05, unitary_metric = 0.99995
Error = :2.16e-03; Runtime: 16.3s; Iterations = 400, grads =   5.703e-06, unitary_metric = 0.99996
Error = :1.23e-03; Runtime: 20.3s; Iterations = 500, grads =   2.610e-06, unitary_metric = 0.99994
Error = :9.99e-04; Runtim

---
## RZ(Θ) Gate: 0.3 ns (at most)
Note: this is the longest any angle seems to take. For some unexpected reason, the gate time is not just linear between 0 and pi.

In [39]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
theta = 0.99*np.pi
U = np.array([[1, 0], [0, np.exp(1j * theta)]])
U = transmon_gate(U, d)

In [40]:
total_time = 0.3
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path = path)

data saved at: /project/ftchong/qoc/pranav/00105_Feb09.h5
Using 4 Taylor terms and 3 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :3.28e-02; Runtime: 0.0s; Iterations = 0, grads =   1.684e-02, unitary_metric = 0.99998
Error = :2.94e-04; Runtime: 0.1s; Iterations = 7, grads =   1.455e-04, unitary_metric = 0.99998
data saved at: /project/ftchong/qoc/pranav/00105_Feb09.h5


---
## RX(Θ) Gate: 4.2 ns (at most)
For this one, the longest time is for theta = pi, as expected.

In [59]:
N = 1
connected_qubit_pairs = []
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
theta = 1.0 * np.pi
U = np.array([[np.cos(theta / 2.0), np.sin(theta / 2.0) * -1j], [np.sin(theta / 2.0) * -1j, np.cos(theta / 2.0)]])
U = transmon_gate(U, d)

In [60]:
total_time = 4.2
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path = path)

data saved at: /project/ftchong/qoc/pranav/00121_Feb09.h5
Using 7 Taylor terms and 2 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :1.00e+00; Runtime: 0.1s; Iterations = 0, grads =   2.069e-05, unitary_metric = 0.99999
Error = :2.57e-02; Runtime: 2.3s; Iterations = 100, grads =   6.163e-04, unitary_metric = 0.99999
Error = :9.83e-03; Runtime: 4.3s; Iterations = 200, grads =   1.356e-04, unitary_metric = 0.99998
Error = :1.25e-02; Runtime: 6.4s; Iterations = 300, grads =   1.881e-04, unitary_metric = 0.99998
Error = :1.11e-02; Runtime: 8.5s; Iterations = 400, grads =   1.852e-04, unitary_metric = 0.99998
Error = :5.56e-03; Runtime: 10.5s; Iterations = 500, grads =   8.018e-05, unitary_metric = 0.99998
Error = :3.04e-03; Runtime: